In [56]:
from gurobipy import *
import networkx as nx
import math
import re
import pandas as pd

In [57]:
def data_load(fileName):
    listSet = []
    list_temp = []
    start_flag = False
    with open(fileName, "r") as file:
        for line in file:
            if line.find('Days') > -1:
                days = int(line.split(":")[1])
            if line.find('Periods_per_day') > -1:
                periods = int(line.split(":")[1])
            if line.strip() == "":
                if list_temp:
                    listSet.append(list_temp.copy())
                    # print(list_temp)
                    list_temp.clear()
                start_flag = True;
                continue
            if start_flag == True:
                if line.find(":") > -1:
                    continue
                else:
                    list_temp.append(line.split())

    df_courses = pd.DataFrame(listSet[0], columns=['courseID','teacher','countOfLectures','minWorkingDays','countOfStudents'])
    df_courses = df_courses.set_index('courseID', drop=True)
    df_rooms = pd.DataFrame(listSet[1], columns=['roomID', 'capacity'])
    df_rooms = df_rooms.set_index('roomID', drop=True)
    dict_curricula = {}
    df_unavailability_constraints = pd.DataFrame(listSet[3], columns=['courseID','day','dayPeriod'])
    df_unavailability_constraints = df_unavailability_constraints.set_index('courseID', drop=True)

    dict_temp = {}
    for item in listSet[2]:
        for i in range(len(item)):
            if i == 1:
                dict_temp['count'] = item[i]
            if i == 2:
                dict_temp['members'] = item[i:]
        dict_curricula[item[0]] = dict_temp.copy()
        dict_temp.clear()

    return df_courses, df_rooms, dict_curricula, df_unavailability_constraints, days, periods

In [65]:
df_courses, df_rooms, dict_curricula, df_unavailability_constraints, days, periods = data_load('comp02.ctt')

# courses K
courses = df_courses.index.to_list()

# rooms R
rooms = df_rooms.index.to_list()

# time_slots T
time_slots = []
for i in range(days):
   for j in range(periods):
       time_slots.append((i,j))

# courses can be take place in which rooms K_r
courses_rooms_pairs = {}
set_temp = set()
for k in courses:
   for r in rooms:
       if int(df_courses.loc[k]['countOfStudents']) <= int(df_rooms.loc[r]['capacity']):
           set_temp.add(r)
   courses_rooms_pairs[k] = set_temp.copy()
   set_temp.clear()

# for rooms which course can be take place in R_r
rooms_courses_pairs = {}
set_temp.clear()
for r in rooms:
    for k in courses:
        if int(df_courses.loc[k]['countOfStudents']) <= int(df_rooms.loc[r]['capacity']):
            set_temp.add(k)
    rooms_courses_pairs[r] = set_temp.copy()
    set_temp.clear()

df_courses[[countOfStudents]] = df_courses[[countOfStudents]].astype(int)
df_courses[df_courses['countOfStudents'] <= 42]
# rooms_courses_pairs

TypeError: cannot convert the series to <class 'int'>